## Reduced Memory Multi-Pass (RMMP) Algorithm
*"Algorithms for Sparse Linear Classifiers in the Massive Data Setting"*, 2008 

**modified shooting algorithm**

- goal: $\beta$ that satisfies $ \max_\beta (\beta ^ T \Psi \beta + \beta ^ T \theta - \gamma ||\beta||_1) $

- "The vector $\Omega$ in the algorithm is defined as $\Omega = 2 \Psi ' \beta + \theta$, where $\Psi '$ is the matrix $\Psi$ with its diagonal entries set to zero. This vector is related to the gradient of the differentiable part of the objective function and consequently can be used for optimality checking."

- "While one can think of numerous stopping criteria for the algorithm, in this paper we stop when successive iterates are sufficiently close to each other (relatively, and with respect to the L2). 
More precisely, we declare convergence whenever
$||\beta_i - \beta_{i-1}||_2 / ||\beta_{i-1}||_2$
is less than some user specified tolerance. Note that $\beta_i$ is the parameter vector at iteration $i$, which is obtained after cycling through and updating all $d$ components once.


![modified shooting pseudocode](images/shooting-pseudocode.png)

In [ ]:
import numpy as np

# ||Bi - Bi-1|| / ||Bi-1|| < tolerance
def has_converged(new_parameters, parameters, tolerance=1e-6):
    return (np.linalg.norm(new_parameters - parameters) / np.linalg.norm(parameters)) < tolerance


def modified_shooting(parameters, d, vector_stats, matrix_stats, tolerance=1e-6):
    # initialize new_parameters
    new_parameters = np.zeros_like(parameters)
    
    # create a new matrix_stats with the diagonals set to 0
    new_matrix_stats = matrix_stats.copy()
    new_matrix_stats[np.diag_indices(d)] = 0
    
    # initialize gradient_vector
    gradient_vector = np.zeros(d)
    
    while True:
        for j in range(d):
            # update new_parameters based on tolerance
            if(abs(gradient_vector[j]  <= tolerance)):
                new_parameters[j] = 0
            elif gradient_vector[j] > tolerance:
                new_parameters[j] = (tolerance - gradient_vector[j]) / (2 * matrix_stats[j, j])
            elif gradient_vector[j] < -tolerance:
                new_parameters[j] = (- tolerance - gradient_vector[j]) / (2 * matrix_stats[j, j])
            
            # update gradient_vector
            gradient_vector = 2 * new_matrix_stats @ new_parameters + vector_stats
        
        # check for convergence
        if has_converged(new_parameters, parameters, tolerance):
            break
        
        # update parameters
        parameters = new_parameters.copy()
    return new_parameters

**the reduced memory multi-pass (RMMP) algorithm**
![rmmp pseudocode](<images/rmmp-pseudocode.png>)

for $y_i=1$

![ai for yi=1](images/ai-for-yi-1.png)

![bi for yi=1](images/bi-for-yi-1.png)

for $y_i=0$

![ai and bi for yi=0](images/ai-bi-for-yi-0.png)

$c$^ $= \beta_{i-1}^T x_i$

$\Phi$ is the link function, either logistic or probit.

In [ ]:
# logistic link function
def logit_model(x):
    return 1 / (1 + np.exp(-x))

def logit_model_derivative(x):
    return logit_model(x) * (1 - logit_model(x))

def logit_model_double_derivative(x):
    return logit_model_derivative(x) * (1 - 2 * logit_model(x))


# quadratic approximation to term likelihood at parameters
def quad_approximation(xi, yi, parameters, d):
    # initialize ai and bi
    ai = np.zeros((d, d))
    bi = np.zeros(d)
    
    c = parameters.T @ xi
    
    if yi == 1:
        ai = 0.5 * (logit_model_double_derivative(c) / logit_model(c) 
                    - (logit_model_derivative(c) / logit_model(c)) **2
                    )
        bi = logit_model_derivative(c) / logit_model(c) - c * ai
    
    elif yi == 0:
        ai = 0.5 * (logit_model_double_derivative(c) / (1 - logit_model(c)) - (logit_model_derivative(c) / (1 - logit_model(c)))**2)
        bi = logit_model_derivative(c) / (1 - logit_model(c)) + c * ai
    
    return ai, bi



def rmmp(X, y, tolerance=1e-6):
    # number of examples and dimension
    t, d = X.shape
    
    parameters = np.zeros(d) # parameters of the regression model
    active_set = set() # components that are either non-zero and optimal or not optimal
    counter = 1 
    gradient_vector = np.zeros(d)
    
    while True:
        vector_stats = np.zeros(d)
        matrix_stats = np.zeros((d, d)) 
        
        for i in range(t):
            # get ith observation (xi, yi)
            xi = X[i].reshape(-1, 1)
            yi = y[i]
            
            ai, bi = quad_approximation(xi, yi, parameters, d)
            
            matrix_stats += ai * (xi @ xi.T)
            vector_stats += bi * xi
            
            new_matrix_stats = matrix_stats.copy()
            new_matrix_stats[np.diag_indices(d)] = 0
            gradient_vector = 2 * new_matrix_stats @ new_parameters + vector_stats
            
        new_parameters = modified_shooting(parameters, d, vector_stats, matrix_stats, tolerance)
        active_set = {j for j in range(d) if gradient_vector[j] >= tolerance}
        
        if has_converged(new_parameters, parameters):
            break
        
        parameters = new_parameters
        counter += 1
    
    return parameters, active_set
            
